# Emotion Detection
## We will Build a neural network thatdetects human emotions from DEAP Datasets.

<div style="text-align: center;">
    <img src="https://www.mdpi.com/sensors/sensors-22-08467/article_deploy/html/images/sensors-22-08467-g001.png" width="800" style="border-radius: 10px;"/>
</div>


### Prerequisites


You'mne v1argparse, os, keras PyTorch, NumPy, machpickle, vision, and IPython in order to run this notebook on your machine.

## Emotion Detection using Autoencoders and LSTM Models

Emotion detection leveraging Autoencoders and LSTM models combines feature extraction and sequence modeling to identify emotional states from input data such as text, audio, or physiological signals.

## Autoderooder

<div style="text-align: center;">
    <img src="https://www.assemblyai.com/blog/content/images/2022/01/autoencoder_architecture.png" width="800" style="border-radius: 10px;"/>
</div>


The **Autoencoder** serves as the feature extractor, capturing meaningful representations of the input data by encoding it into a compressed latent space. 

- The **Encoder** processes the input through a series of dense or convolutional layers (depending on the data type), reducing its dimensionality while preserving critical patterns. 
- The **Decoder** reconstructs the original input from the latent space to ensure the encoded features are informative and accurate.

## LSTM (Long Short-Term Memory)

<div style="text-align: center;">
    <img src="https://cdn.dida.do/new-project-3-1-1024x607-1024x585.webp" width="800" style="border-radius: 10px;"/>
</div>


The **LSTM** model is responsible for sequence modeling. 

- It processes sequential input, such as time-series data or sentence structures, by leveraging its ability to learn long-term dependencies. 
- The LSTM network operates on the features extracted by the Autoencoder, analyzing the temporal or contextual relationships to predict the corresponding emotional states.

## Training Process

1. **Autoencoder Training**: The Autoencoder is trained to minimize reconstruction error, ensuring high-quality feature extraction.
2. **LSTM Training**: The LSTM model is fine-tuned using the features extracted by the Autoencoder to minimize emotion classification or regression loss.

By combining the strengths of both components, this architecture effectively detects and categorizes emotions, providing robust performance across diverse applications like:

- Sentiment analysis
- Affective computing

## Loading Libraries and DEAP Dataset

For emotion detection, we require a reliable dataset containing physiological signals and corresponding emotional labels. We'll use the **DEAP dataset**, a well-known resource for affective computing research, which includes EEG and peripheral physiological signals recorded from participants while watching videos.

"[DEAP: A Dataset for Emotion Analysis using Physiological Signals](http://www.eecs.qmul.ac.uk/mmv/datasets/deap/)"

### Libraries and Dataset Preparation

We'll begin by importing essential libraries such as TensorFlow, PyTorch, or other machine learning frameworks, along with tools for processing physiological signals. The DEAP dataset can be loaded using libraries like `numpy` and `scipy` to handle the provided `.mat` or `.bdf` files. Data preprocessing steps typically include:

1. **Signal Filtering**: Applying band-pass filters to clean noise from raw EEG and physiological data.
2. **Feature Extraction**: Extracting meaningful features such as power spectral density, time-domain features, or statistical metrics from the signals.
3. **Label Mapping**: Mapping each signal to its corresponding emotional labels, such as valence, arousal, or dominance.

The DEAP dataset is suitable for training models to detect emotions by leveraging both the raw physiological signals and the extracted features.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input
/kaggle/input/data_preprocessed_python
/kaggle/input/Metadata
/kaggle/input/audio_stimuli_MIDI
/kaggle/input/metadata_xls
/kaggle/input/audio_stimuli_MIDI_tempo24


In [2]:
!pip install -q mne==1.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 68.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: mne
    Found existing installation: mne 1.8.0
    Uninstalling mne-1.8.0:
      Successfully uninstalled mne-1.8.0


In [3]:
!pip install -q tensorflow==2.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 100.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 106.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 38.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-es

In [4]:
from IPython.utils import io
import numpy as np
import numpy
import collections

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle

import scipy.io
from scipy import signal, integrate
import matplotlib.pyplot as plt

import keras
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Dropout
from tensorflow.keras.models import load_model

import mne
import pickle
import math
from sklearn.model_selection import train_test_split

n_second = 60
n_segment = 2*n_second-1
n_points = n_second*128
bottleneck = 12

2024-11-28 13:37:52.379571: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutex6unlockEv']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by

In [5]:
# Computes the power spectrum of signal X over specified frequency bands Band using the Fourier Transform and sums the power within each band.
# Returns the power in each band and the power ratio (normalized power distribution) across all bands.
def bin_power(X, Band, Fs):
    C = numpy.fft.fft(X)
    C = abs(C)
    Power = numpy.zeros(len(Band) - 1)
    for Freq_Index in range(0, len(Band) - 1):
        Freq = float(Band[Freq_Index])
        Next_Freq = float(Band[Freq_Index + 1])
        Power[Freq_Index] = sum(
            C[int(numpy.floor(Freq / Fs * len(X))):
                int(numpy.floor(Next_Freq / Fs * len(X)))]
        )
    Power_Ratio = Power / sum(Power)
    return Power, Power_Ratio


# Calculates the entropy of the power distribution in the frequency bands, representing the disorder or complexity of the signal.
# Uses the bin_power function to compute power ratios if not provided, and returns the negative entropy value for each band.
def spectral_entropy(X, Band, Fs, Power_Ratio=None):
    if Power_Ratio is None:
        Power, Power_Ratio = bin_power(X, Band, Fs)

    Spectral_Entropy = numpy.zeros(len(Power_Ratio))
    for i in range(0, len(Power_Ratio)):
        Spectral_Entropy[i] = Power_Ratio[i] * numpy.log(Power_Ratio[i])
    # to save time, minus one is omitted
    return -1 * Spectral_Entropy

### Normalization and Standarization

In [6]:
# build in
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
scaler = MinMaxScaler().fit(a)
a = scaler.transform(a)
print(a)

#custom
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
max_, min_ = np.amax(a), np.amin(a)
a = (a - min_) / (max_ - min_)
print(a)

def normalise_2D(a, multiple):
    max_, min_ = np.amax(a), np.amin(a)
    a = (a - min_) / (max_ - min_)
    return multiple*a

a = np.array([[1,2,3],[4,5,6],[7,8,9]])
normalise_2D(a, 1) # range -10 to 10

[[0.  0.  0. ]
 [0.5 0.5 0.5]
 [1.  1.  1. ]]
[[0.    0.125 0.25 ]
 [0.375 0.5   0.625]
 [0.75  0.875 1.   ]]


array([[0.   , 0.125, 0.25 ],
       [0.375, 0.5  , 0.625],
       [0.75 , 0.875, 1.   ]])

In [7]:
# build in
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
scaler = StandardScaler().fit(a)
a = scaler.transform(a)
print(a)

# custom
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
std = np.std(a)
mean = np.mean(a)
a = (a-mean)/std
print(a)

def standardise_2D(a, multiple):
    std = np.std(a)
    mean = np.mean(a)
    a = (a-mean)/std
    return multiple*a

a = np.array([[1,2,3],[4,5,6],[7,8,9]])
standardise_2D(a, 10)

[[-1.22474487 -1.22474487 -1.22474487]
 [ 0.          0.          0.        ]
 [ 1.22474487  1.22474487  1.22474487]]
[[-1.54919334 -1.161895   -0.77459667]
 [-0.38729833  0.          0.38729833]
 [ 0.77459667  1.161895    1.54919334]]


array([[-15.49193338, -11.61895004,  -7.74596669],
       [ -3.87298335,   0.        ,   3.87298335],
       [  7.74596669,  11.61895004,  15.49193338]])

### Data Loading And Preprocessing
#### The idea it to create 2 datasets, one for arousal and another for valence to create hybrid models for both

In [8]:
def convertOneData(file_name):
    with open(file_name, 'rb') as file:
        mat = pickle.load(file, encoding='latin1')
        labels = mat['labels'][:, 0:2] # only valence, arousal, no dominance, liking
        data = mat['data'][:, 0:32, 3*128:] # only first 32 channels ['Fp1','AF3','F3','F7','FC5',
        #'FC1','C3','T7','CP5','CP1','P3','P7','PO3','O1','Oz','Pz','Fp2','AF4','Fz','F4',
        #'F8','FC6','FC2','Cz','C4','T8','CP6','CP2','P4','P8','PO4','O2']
        #and skip first 3 seconds
        #print(labels.shape, data.shape) # (40, 2) (40, 32, 8064)
        valence_labels, valence_data = [], []
        arousal_labels, arousal_data = [], []
        for i, label in enumerate(labels):
            valence, arousal = label[0], label[1]
            if valence > 5.5: # value >5.5 is high
                valence_labels.append(1)
                valence_data.append(data[i])
            if valence < 4.5: # value <4.5 is high
                valence_labels.append(0)
                valence_data.append(data[i])
            if arousal > 5.5:
                arousal_labels.append(1)
                arousal_data.append(data[i])
            if arousal < 4.5:
                arousal_labels.append(0)
                arousal_data.append(data[i])
        
        print("valence: ", len(valence_labels), "arousal: ", len(arousal_labels))
    return valence_labels, valence_data, arousal_labels, arousal_data        
    

def convertAllData():
    all_valence_labels, all_valence_data = [], []
    all_arousal_labels, all_arousal_data = [], []
    for i in range(32): 
        if i < 10: # subject 01-09
            name = '%0*d' % (2,i+1)
        else: # subject 10-32
            name = i+1
        file_name = "/kaggle/input/data_preprocessed_python/s"+str(name)+".dat"
        print(file_name)
        valence_labels, valence_data, arousal_labels, arousal_data = convertOneData(file_name) # convert one subject data
        
        all_valence_labels += valence_labels
        for valence_d in valence_data: # each trial
            valence_d = standardise_2D(valence_d, 1)
            all_valence_data.append(valence_d)    
        all_arousal_labels += arousal_labels
        for arousal_d in arousal_data:
            arousal_d = standardise_2D(arousal_d, 1)
            all_arousal_data.append(arousal_d)
     
    all_valence_labels = np.array(all_valence_labels)
    all_valence_data = np.array(all_valence_data)
    all_arousal_labels = np.array(all_arousal_labels)
    all_arousal_data = np.array(all_arousal_data)
    print("Valence trial data for all subject: ", all_valence_labels.shape,all_valence_data.shape)
    print("Arousal trial data for all subject: ", all_arousal_labels.shape,all_arousal_data.shape)
    # save numpy array of total data to files
    np.save('/kaggle/working/Data/processed_DEAP/valence/' + 'all_valence_labels.npy', all_valence_labels)
    np.save('/kaggle/working/Data/processed_DEAP/valence/' + 'all_valence_data.npy', all_valence_data)
    np.save('/kaggle/working/Data/processed_DEAP/arousal/' + 'all_arousal_labels.npy', all_arousal_labels)
    np.save('/kaggle/working/Data/processed_DEAP/arousal/' + 'all_arousal_data.npy', all_arousal_data)

In [9]:
!mkdir /kaggle/working/Data/
!mkdir /kaggle/working/Data/processed_DEAP/
!mkdir /kaggle/working/Data/processed_DEAP/arousal/
!mkdir /kaggle/working/Data/processed_DEAP/valence/
!mkdir /kaggle/working/Results/
!mkdir /kaggle/working/Results/autoencoder_model/
!mkdir /kaggle/working/Results/LSTM_model/

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [10]:
convertAllData()

In [17]:
def load_np_data(dimension):
    if dimension == 'valence':
        all_labels, all_data = np.load('/kaggle/working/Data/processed_DEAP/valence/' + 'all_valence_labels.npy', allow_pickle=True), np.load('/kaggle/working/Data/processed_DEAP/valence/' + 'all_valence_data.npy', allow_pickle=True)
        print("Total valence: ", all_labels.shape, all_data.shape)
        #print("High and low valence: ", collections.Counter(all_labels))# 587 high valence, 472 low valence
    elif dimension == 'arousal':
        all_labels, all_data = np.load('/kaggle/working/Data/processed_DEAP/arousal/' + 'all_arousal_labels.npy', allow_pickle=True), np.load('/kaggle/working/Data/processed_DEAP/arousal/' + 'all_arousal_data.npy', allow_pickle=True)
        print("Total arousal: ", all_labels.shape, all_data.shape)
        #print("High and low arousal: ", collections.Counter(all_labels))# 620 high arousal, 462 low arousal
    return all_labels, all_data

#### Loading data for each model

In [18]:
all_labels, all_data = load_np_data(dimension="valence")
#all_labels, all_data = load_np_data(dimension="arousal")

Total valence:  (1059,) (1059, 32, 7680)


In [19]:
# after standardised
print(np.amax(all_data)) # max value
print(np.amin(all_data)) # min value

# print(np.amax(all_valence_data[0])) # max value
# print(np.amin(all_valence_data[0])) # min value

91.75265724740024
-100.00674912321686


#### Feature Extraction
We will compute the Power Spectral Density (PSD) of EEG data across frequency bands for all channels and time segments using Welch’s method, and reshapes the PSD for integration-based band power calculation.
Integrating PSD values over specific frequency bands (e.g., delta, theta, alpha, beta) for each segment and channel, returning a flattened feature matrix representing band power.

In [20]:
def trial_psd_extraction_integration(data): # data shape (12, 8064)
    info = mne.create_info(ch_names= ['1','2','3','4','5','6','7','8','9','10','11','12'], sfreq=128);
    raw = mne.io.RawArray(data, info, first_samp=0, copy='auto', verbose=None);
    psd_origin, f = mne.time_frequency.psd_welch(raw, fmin=0, fmax=60, n_fft=128, n_overlap=64, n_per_seg=128, picks='all', window='hann', average=None, verbose=None)# average='mean' or None
    # print(psd_origin.shape, f.shape) # (12, 61, 125) (61,) 61 frequency
    psd = np.moveaxis(psd_origin, -1, 0) # (125, 12, 61)
    # calculate frequency band power using integration
    band_power = [] # band power for all segments
    for segment in psd:
        segment_band_power = [] # band power for all channels in one segment
        for psd_channel in segment:
            y_int = integrate.cumulative_trapezoid(psd_channel, f, initial=0) # integrate to calculate band power
            one_band_power = np.array([y_int[7]-y_int[4],y_int[13]-y_int[8],y_int[30]-y_int[14],y_int[51]-y_int[31]])
            segment_band_power.append(one_band_power)
        band_power.append(segment_band_power)
    band_power = np.array(band_power) # (125, 12, 4)
    band_power = np.moveaxis(band_power, -1, 1) # (125, 4, 12)
    band_power = band_power.reshape((n_segment, bottleneck*4)) # flatten feature (125, 48)
    band_power = 10*band_power
    return band_power

#### Vector transformations for ecoding and decoding

In [21]:
# change dimension from (849, 32, 7680)to (849, 8064, 32) then to (6846336, 32) for input 32-dimension vector to autoencoder
def vector_transform(data):
    vectors = np.moveaxis(data, 1, -1)
    vectors = vectors.reshape((vectors.shape[0]*vectors.shape[1], vectors.shape[2]))
    return vectors

# change output of autoencoder dimension from (6846336, 12) to (849, 8064, 12) then to (849, 12, 8064)
def inverse_vector_transform(vectors):
    data = vectors.reshape((int(vectors.shape[0]/n_points), n_points, vectors.shape[1]))
    data = np.moveaxis(data, -1, 1)
    return data

# **Multiple folds/models training (AutoEncoder + LSTM)**

In [22]:
all_data, all_labels = shuffle(all_data, all_labels, random_state=0)
n = len(all_labels) # 1059
print(n)
fold_n = math.floor(n/10) # 105
print(fold_n)
all_data, all_labels = all_data[:10*fold_n], all_labels[:10*fold_n] # (1050, 32, 8064)  
print(all_data.shape)

1059
105
(1050, 32, 7680)


In [23]:
def process(test_fold_number):
    # train has 9 folds, test has 1 fold
    train_data = np.concatenate((all_data[:test_fold_number*fold_n], all_data[fold_n+test_fold_number*fold_n:]), axis=0)
    train_labels = np.concatenate((all_labels[:test_fold_number*fold_n], all_labels[fold_n+test_fold_number*fold_n:]), axis=0)
    test_data = all_data[test_fold_number*fold_n : fold_n+test_fold_number*fold_n]
    test_labels = all_labels[test_fold_number*fold_n : fold_n+test_fold_number*fold_n]
    print(train_data.shape,test_data.shape) # (945, 32, 8064) (105, 32, 8064)
    
    # change dimension to 32-dimension vector for input to autoencoder
    train_vectors = vector_transform(train_data)
    test_vectors = vector_transform(test_data)
    print(train_vectors.shape, test_vectors.shape)# (7620480, 32) (846720, 32)
    
    # -------- Create new autoencoder --------
    input_layer = Input(shape=(32,))
    encoded = Dense(64, activation=None)(input_layer)
    bottleneck_layer = Dense(bottleneck, activation=None)(encoded)
    decoded = Dense(64, activation=None)(bottleneck_layer)
    decoded = Dense(32, activation=None)(decoded)
    autoencoder = Model(input_layer, decoded)
    autoencoder.summary()

    encoder = Model(input_layer, bottleneck_layer)
    encoder.summary()

    decoder_input_layer = Input(shape=(bottleneck,))
    decoder_layer = autoencoder.layers[-2](decoder_input_layer)
    decoder_layer = autoencoder.layers[-1](decoder_layer)
    decoder = Model(decoder_input_layer, decoder_layer)
    decoder.summary()
    
    # -------- Compile and train autoencoder --------
    autoencoder.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])
    autoencoder.fit(train_vectors, train_vectors, epochs=1, batch_size=64, shuffle=True, validation_data=(test_vectors, test_vectors))
    autoencoder.save("/kaggle/working/Results/autoencoder_model/autoencoder_model_test_fold_" + str(test_fold_number) + ".h5")
    
    # -------- Encode train and test data by pass through encoder --------
    train_data_encoded = encoder.predict(train_vectors)
    train_data_encoded = inverse_vector_transform(train_data_encoded) 
    test_data_encoded = encoder.predict(test_vectors) 
    test_data_encoded = inverse_vector_transform(test_data_encoded) 
    print("Encoded training data shape: ", train_data_encoded.shape)
    print("Encoded test data shape: ", test_data_encoded.shape)
    
    # -------- Feature extraction from 12 source signal --------
    train_band_power = [] # band power feature sequence for train trials
    for data in train_data_encoded: # for every train trial
        with io.capture_output() as captured:
            trial_band_power = trial_psd_extraction_integration(data) # data shape (12, 8064)
        train_band_power.append(trial_band_power)
    train_band_power = np.array(train_band_power)
    
    test_band_power = [] # band power feature sequence for test trials
    for data in test_data_encoded: # for every test trial
        with io.capture_output() as captured:
            trial_band_power = trial_psd_extraction_integration(data) # data shape (12, 8064)
        test_band_power.append(trial_band_power)
    test_band_power = np.array(test_band_power)
    print("All features of training data shape: ", train_band_power.shape) # shape (849, 125, 48)
    print("All features of test data shape: ", test_band_power.shape) # shape (95, 125, 48)
    
    # -------- Create new LSTM model --------
    x=Input(shape=(n_segment,bottleneck*4)) # flatten (12,4) to 48
    x1=LSTM(n_segment)(x)
    x2=Dense(n_segment)(x1)
    x3=Dense(12)(x2)
    output=Dense(1, activation="sigmoid")(x2)
    model=Model(x, output)

    # -------- Compile and train LSTM --------
    model.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])
    history = model.fit(train_band_power, train_labels, epochs=30, batch_size=8, validation_data=(test_band_power, test_labels))
    print("Hightest accuracy: " + str(max(history.history['val_accuracy'])))
    model.save("/kaggle/working/Results/LSTM_model/LSTM_model_test_fold_" + str(test_fold_number) + ".h5")

In [26]:
for i in range(10):
    print("********** Test Fold " + str(i) + " ************")
    process(i)

********** Test Fold 0 ************
(945, 32, 7680) (105, 32, 7680)
(7257600, 32) (806400, 32)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 12)                780       
                                                                 
 dense_2 (Dense)             (None, 64)                832       
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
Total params: 5,804
Trainable params: 5,804
Non-trainable params: 0
______________________________

# **Full data training on one model**

## **Valence Model**

In [24]:
all_labels, all_data = load_np_data(dimension="valence")

Total valence:  (1059,) (1059, 32, 7680)


### **Tranformers**

In [25]:
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np

# Shuffle and prepare all data
all_data, all_labels = shuffle(all_data, all_labels, random_state=0)
print(all_data.shape, all_labels.shape)

# Transform all data to vector form
all_vectors = vector_transform(all_data)
print(all_vectors.shape)

# -------- Create new autoencoder --------
input_layer = Input(shape=(32,))
encoded = Dense(64, activation=None)(input_layer)
bottleneck_layer = Dense(bottleneck, activation=None)(encoded)
decoded = Dense(64, activation=None)(bottleneck_layer)
decoded = Dense(32, activation=None)(decoded)
autoencoder = Model(input_layer, decoded)
autoencoder.summary()

encoder = Model(input_layer, bottleneck_layer)
encoder.summary()

decoder_input_layer = Input(shape=(bottleneck,))
decoder_layer = autoencoder.layers[-2](decoder_input_layer)
decoder_layer = autoencoder.layers[-1](decoder_layer)
decoder = Model(decoder_input_layer, decoder_layer)
decoder.summary()

# -------- Compile and train autoencoder --------
autoencoder.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])
autoencoder.fit(all_vectors, all_vectors, epochs=1, batch_size=64, shuffle=True, validation_split=0.1)
autoencoder.save("/kaggle/working/Results/autoencoder_model/autoencoder_model_arousal.h5")

# -------- Encode all data --------
all_data_encoded = encoder.predict(all_vectors)
all_data_encoded = inverse_vector_transform(all_data_encoded)
print("Encoded data shape: ", all_data_encoded.shape)

# -------- Feature extraction from 12 source signals --------
all_band_power = []
for data in all_data_encoded:  # For every trial
    with io.capture_output() as captured:
        trial_band_power = trial_psd_extraction_integration(data)
    all_band_power.append(trial_band_power)
all_band_power = np.array(all_band_power)
print("All features shape: ", all_band_power.shape)

# -------- Train-test split --------
train_data, test_data, train_labels, test_labels = train_test_split(all_band_power, all_labels, test_size=0.1, random_state=42)
print(train_data.shape, test_data.shape)

# -------- Create Transformer model --------
def transformer_encoder(inputs, num_heads, ff_dim, dropout_rate):
    # Multi-head Self-Attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    # Add & Normalize
    attention_output = LayerNormalization(epsilon=1e-6)(inputs + attention_output)

    # Feed-Forward Network
    ff_output = Dense(ff_dim, activation="relu")(attention_output)
    ff_output = Dense(inputs.shape[-1])(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)

    # Add & Normalize
    return LayerNormalization(epsilon=1e-6)(attention_output + ff_output)

# Input to the Transformer
inputs = Input(shape=(train_data.shape[1], train_data.shape[2]))

# Transformer block
x = transformer_encoder(inputs, num_heads=4, ff_dim=128, dropout_rate=0.1)
x = GlobalAveragePooling1D()(x)  # Pooling to reduce sequence dimension
x = Dense(64, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

# Model definition
transformer_model = Model(inputs, output)
transformer_model.summary()

# -------- Compile and train Transformer model --------
transformer_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = transformer_model.fit(
    train_data, train_labels, 
    epochs=30, 
    batch_size=16, 
    validation_data=(test_data, test_labels)
)
print("Highest accuracy: " + str(max(history.history['val_accuracy'])))
transformer_model.save("/kaggle/working/Results/Transformer_model/Transformer_model_arousal.h5")


(1059, 32, 7680) (1059,)
(8133120, 32)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 12)                780       
                                                                 
 dense_2 (Dense)             (None, 64)                832       
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
Total params: 5,804
Trainable params: 5,804
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
___

### **Transformer + LSTM**

In [28]:
inputs = Input(shape=(train_data.shape[1], train_data.shape[2]))
x = LSTM(64, return_sequences=True)(inputs)
x = transformer_encoder(x, num_heads=4, ff_dim=128, dropout_rate=0.1)
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)
hybrid_model = Model(inputs, output)

hybrid_model.summary()

# -------- Compile and train Transformer model --------
hybrid_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = hybrid_model.fit(
    train_data, train_labels, 
    epochs=30, 
    batch_size=16, 
    validation_data=(test_data, test_labels)
)
print("Highest accuracy: " + str(max(history.history['val_accuracy'])))
hybrid_model.save("/kaggle/working/Results/Transformer_model/Transformer_hybrid_model_arousal.h5")


Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 119, 48)]    0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 119, 64)      28928       ['input_7[0][0]']                
                                                                                                  
 multi_head_attention_2 (MultiH  (None, 119, 64)     66368       ['lstm_1[0][0]',                 
 eadAttention)                                                    'lstm_1[0][0]']                 
                                                                                                  
 dropout_4 (Dropout)            (None, 119, 64)      0           ['multi_head_attention_2[0]

In [31]:
import tensorflow as tf

def positional_encoding(sequence_length, model_dim):
    position = np.arange(sequence_length)[:, np.newaxis]
    div_term = np.exp(np.arange(0, model_dim, 2) * -(np.log(10000.0) / model_dim))
    pos_encoding = np.zeros((sequence_length, model_dim))
    pos_encoding[:, 0::2] = np.sin(position * div_term)
    pos_encoding[:, 1::2] = np.cos(position * div_term)
    return tf.constant(pos_encoding, dtype=tf.float32)

# Apply positional encoding
inputs = Input(shape=(train_data.shape[1], train_data.shape[2]))
pos_encoding = positional_encoding(train_data.shape[1], train_data.shape[2])
x = inputs + pos_encoding
x = LSTM(64, return_sequences=True)(inputs)
x = transformer_encoder(x, num_heads=4, ff_dim=128, dropout_rate=0.1)
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)
hybrid_model = Model(inputs, output)

hybrid_model.summary()

# -------- Compile and train Transformer model --------
hybrid_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = hybrid_model.fit(
    train_data, train_labels, 
    epochs=30, 
    batch_size=32, 
    validation_data=(test_data, test_labels)
)
print("Highest accuracy: " + str(max(history.history['val_accuracy'])))
hybrid_model.save("/kaggle/working/Results/Transformer_model/Transformer_hybrid_model_arousal.h5")


Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 119, 48)]    0           []                               
                                                                                                  
 lstm_3 (LSTM)                  (None, 119, 64)      28928       ['input_10[0][0]']               
                                                                                                  
 multi_head_attention_4 (MultiH  (None, 119, 64)     66368       ['lstm_3[0][0]',                 
 eadAttention)                                                    'lstm_3[0][0]']                 
                                                                                                  
 dropout_8 (Dropout)            (None, 119, 64)      0           ['multi_head_attention_4[0

### **LSTM + Transformer + Additive Fusion** (Combines the LSTM output and the Transformer output using the Add layer)

In [37]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, GlobalAveragePooling1D, Add


# Ensure TensorFlow is using the GPU
def setup_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

setup_gpu()

def positional_encoding(sequence_length, model_dim):
    position = np.arange(sequence_length)[:, np.newaxis]
    div_term = np.exp(np.arange(0, model_dim, 2) * -(np.log(10000.0) / model_dim))
    pos_encoding = np.zeros((sequence_length, model_dim))
    pos_encoding[:, 0::2] = np.sin(position * div_term)
    pos_encoding[:, 1::2] = np.cos(position * div_term)
    return tf.constant(pos_encoding, dtype=tf.float32)

# Apply positional encoding
inputs = Input(shape=(train_data.shape[1], train_data.shape[2]))
pos_encoding = positional_encoding(train_data.shape[1], train_data.shape[2])
x = inputs + pos_encoding
x_lstm = LSTM(128, return_sequences=True)(inputs)
x_transformer = transformer_encoder(x_lstm, num_heads=4, ff_dim=128, dropout_rate=0.1)
x = Add()([x_lstm, x_transformer])
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)
hybrid_model = Model(inputs, output)

hybrid_model.summary()

# -------- Compile and train Transformer model --------

optimizer = Adam(learning_rate=0.0001)
hybrid_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Optionally, specify the GPU device (if multiple GPUs are available)
with tf.device('/device:GPU:0'):  # Use '/device:GPU:1' for a second GPU, etc.
    history = hybrid_model.fit(train_data, train_labels,
                               epochs=40,
                               batch_size=16,
                               validation_data=(test_data, test_labels))

print("Highest accuracy: " + str(max(history.history['val_accuracy'])))
hybrid_model.save("/kaggle/working/Results/Transformer_model/Transformer_hybrid_model_2_arousal.h5")

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 119, 48)]    0           []                               
                                                                                                  
 lstm_9 (LSTM)                  (None, 119, 128)     90624       ['input_16[0][0]']               
                                                                                                  
 multi_head_attention_10 (Multi  (None, 119, 128)    263808      ['lstm_9[0][0]',                 
 HeadAttention)                                                   'lstm_9[0][0]']                 
                                                                                                  
 dropout_20 (Dropout)           (None, 119, 128)     0           ['multi_head_attention_10[

## Arousal Model

In [30]:
all_labels, all_data = load_np_data(dimension="arousal")

Total arousal:  (1082,) (1082, 32, 7680)


In [31]:
# Shuffle and prepare all data
all_data, all_labels = shuffle(all_data, all_labels, random_state=0)
print(all_data.shape, all_labels.shape)

# Transform all data to vector form
all_vectors = vector_transform(all_data)
print(all_vectors.shape)

# -------- Create new autoencoder --------
input_layer = Input(shape=(32,))
encoded = Dense(64, activation=None)(input_layer)
bottleneck_layer = Dense(bottleneck, activation=None)(encoded)
decoded = Dense(64, activation=None)(bottleneck_layer)
decoded = Dense(32, activation=None)(decoded)
autoencoder = Model(input_layer, decoded)
autoencoder.summary()

encoder = Model(input_layer, bottleneck_layer)
encoder.summary()

decoder_input_layer = Input(shape=(bottleneck,))
decoder_layer = autoencoder.layers[-2](decoder_input_layer)
decoder_layer = autoencoder.layers[-1](decoder_layer)
decoder = Model(decoder_input_layer, decoder_layer)
decoder.summary()

# -------- Compile and train autoencoder --------
autoencoder.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])
autoencoder.fit(all_vectors, all_vectors, epochs=1, batch_size=64, shuffle=True, validation_split=0.1)
autoencoder.save("/kaggle/working/Results/autoencoder_model/autoencoder_model_arousal.h5")

# -------- Encode all data --------
all_data_encoded = encoder.predict(all_vectors)
all_data_encoded = inverse_vector_transform(all_data_encoded)
print("Encoded data shape: ", all_data_encoded.shape)

# -------- Feature extraction from 12 source signals --------
all_band_power = []
for data in all_data_encoded:  # For every trial
    with io.capture_output() as captured:
        trial_band_power = trial_psd_extraction_integration(data)
    all_band_power.append(trial_band_power)
all_band_power = np.array(all_band_power)
print("All features shape: ", all_band_power.shape)

(1082, 32, 7680) (1082,)
(8309760, 32)
Model: "model_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 32)]              0         
                                                                 
 dense_77 (Dense)            (None, 64)                2112      
                                                                 
 dense_78 (Dense)            (None, 12)                780       
                                                                 
 dense_79 (Dense)            (None, 64)                832       
                                                                 
 dense_80 (Dense)            (None, 32)                2080      
                                                                 
Total params: 5,804
Trainable params: 5,804
Non-trainable params: 0
_________________________________________________________________
Model: "model_45"

In [32]:
# -------- Train-test split --------
train_data, test_data, train_labels, test_labels = train_test_split(all_band_power, all_labels, test_size=0.1, random_state=42)
print(train_data.shape, test_data.shape)

# -------- Create new LSTM model --------
x = Input(shape=(n_segment, bottleneck * 4))
x1 = LSTM(n_segment)(x)
x2 = Dense(n_segment)(x1)
x3 = Dense(12)(x2)
output = Dense(1, activation="sigmoid")(x2)
model = Model(x, output)

# -------- Compile and train LSTM --------
model.compile(optimizer='SGD', loss='mse', metrics=['accuracy'])
history = model.fit(train_data, train_labels, epochs=30, batch_size=8, validation_data=(test_data, test_labels))
print("Highest accuracy: " + str(max(history.history['val_accuracy'])))
model.save("/kaggle/working/Results/LSTM_model/LSTM_model_arousal.h5")

(973, 119, 48) (109, 119, 48)
Epoch 1/30
122/122 [==============================] - 10s 69ms/step - loss: 0.2466 - accuracy: 0.5591 - val_loss: 0.2292 - val_accuracy: 0.6514
Epoch 2/30
122/122 [==============================] - 8s 62ms/step - loss: 0.2281 - accuracy: 0.6321 - val_loss: 0.2271 - val_accuracy: 0.6514
Epoch 3/30
122/122 [==============================] - 7s 60ms/step - loss: 0.2192 - accuracy: 0.6660 - val_loss: 0.2250 - val_accuracy: 0.6422
Epoch 4/30
122/122 [==============================] - 7s 60ms/step - loss: 0.2127 - accuracy: 0.6814 - val_loss: 0.2227 - val_accuracy: 0.6514
Epoch 5/30
122/122 [==============================] - 7s 61ms/step - loss: 0.2070 - accuracy: 0.6989 - val_loss: 0.2273 - val_accuracy: 0.6330
Epoch 6/30
122/122 [==============================] - 7s 60ms/step - loss: 0.2032 - accuracy: 0.7020 - val_loss: 0.2248 - val_accuracy: 0.6514
Epoch 7/30
122/122 [==============================] - 7s 60ms/step - loss: 0.1978 - accuracy: 0.7143 - val_loss

# Final models tesing

In [34]:
valence_labels, valence_data, arousal_labels, arousal_data = convertOneData("/kaggle/input/data_preprocessed_python/s01.dat")

valence:  38 arousal:  39


In [35]:
arousal_d = standardise_2D(arousal_data, 1)

In [36]:
arousal_d.shape

(39, 32, 7680)

In [37]:
arousal_data_vectors = vector_transform(arousal_d)

In [38]:
encoder_path = "/kaggle/working/Results/autoencoder_model/autoencoder_model_test_fold_0.h5"
lstm_path = "/kaggle/working/Results/LSTM_model/LSTM_model_test_fold_0.h5"

In [39]:
encoder = load_model(encoder_path)
lstm_model = load_model(lstm_path)
encoder_model = Model(encoder.input, encoder.get_layer('dense_1').output)

In [40]:
arousal_data_vectors.shape

(299520, 32)

In [41]:
arousal_encoded = encoder_model.predict(arousal_data_vectors)

9360/9360 [==============================] - 9s 939us/step


In [42]:
arousal_data_inverted = inverse_vector_transform(arousal_encoded)

In [43]:
arousal_features = []
for data in arousal_data_inverted:
    features = trial_psd_extraction_integration(data)
    arousal_features.append(features)
arousal_features = np.array(arousal_features)

predictions = lstm_model.predict(arousal_features)

# Print a few random predictions
for i in range(0, len(predictions)):
    print(f"Prediction: {predictions[i][0]:.2f}, Ground Truth: {arousal_labels[i]}")

Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 dat

# Emotion Detection based on the created models

In [50]:
def map_to_emotion(valence_pred, arousal_pred):
    if valence_pred == 1 and arousal_pred == 1:
        return "Happy"  # High Valence, High Arousal
    elif valence_pred == 1 and arousal_pred == 0:
        return "Content"  # High Valence, Low Arousal
    elif valence_pred == 0 and arousal_pred == 1:
        return "Angry"  # Low Valence, High Arousal
    elif valence_pred == 0 and arousal_pred == 0:
        return "Sad"  # Low Valence, Low Arousal
    else:
        return "Unknown"  # For safety

araousel_encoder_path = "/kaggle/working/Results/autoencoder_model/autoencoder_model_arousal.h5"
valence_encoder_path = "/kaggle/working/Results/autoencoder_model/autoencoder_model_valence.h5"

lstm_valence_path = "/kaggle/working/Results/LSTM_model/LSTM_model_valence.h5"
lstm_araousel_path = "/kaggle/working/Results/LSTM_model/LSTM_model_arousal.h5"

In [46]:
araousel_encoder = load_model(araousel_encoder_path)
araousel_lstm_model = load_model(lstm_araousel_path)

araousel_encoder.summary()
araousel_encoder = Model(araousel_encoder.input, araousel_encoder.get_layer('dense_78').output)

Model: "model_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 32)]              0         
                                                                 
 dense_77 (Dense)            (None, 64)                2112      
                                                                 
 dense_78 (Dense)            (None, 12)                780       
                                                                 
 dense_79 (Dense)            (None, 64)                832       
                                                                 
 dense_80 (Dense)            (None, 32)                2080      
                                                                 
Total params: 5,804
Trainable params: 5,804
Non-trainable params: 0
_________________________________________________________________


In [52]:
valence_encoder = load_model(valence_encoder_path)
valence_lstm_model = load_model(lstm_valence_path)

valence_encoder.summary()
valence_encoder = Model(valence_encoder.input, valence_encoder.get_layer('dense_71').output)

Model: "model_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_31 (InputLayer)       [(None, 32)]              0         
                                                                 
 dense_70 (Dense)            (None, 64)                2112      
                                                                 
 dense_71 (Dense)            (None, 12)                780       
                                                                 
 dense_72 (Dense)            (None, 64)                832       
                                                                 
 dense_73 (Dense)            (None, 32)                2080      
                                                                 
Total params: 5,804
Trainable params: 5,804
Non-trainable params: 0
_________________________________________________________________


In [53]:
valence_labels, valence_data, arousal_labels, arousal_data = convertOneData("/kaggle/input/data_preprocessed_python/s01.dat")

valence:  38 arousal:  39


In [54]:
valence_d = standardise_2D(valence_data, 1)
valence_data_vectors = vector_transform(valence_d)
valence_encoded = valence_encoder.predict(valence_data_vectors)

valence_data_inverted = inverse_vector_transform(valence_encoded)
valence_features = [trial_psd_extraction_integration(data) for data in valence_data_inverted]
valence_features = np.array(valence_features)
valence_predictions = valence_lstm_model.predict(valence_features)

9120/9120 [==============================] - 9s 929us/step
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective

In [55]:
arousal_d = standardise_2D(arousal_data, 1)
arousal_data_vectors = vector_transform(arousal_d)
arousal_encoded = araousel_encoder.predict(arousal_data_vectors)

arousal_data_inverted = inverse_vector_transform(arousal_encoded)
arousal_features = [trial_psd_extraction_integration(data) for data in arousal_data_inverted]
arousal_features = np.array(arousal_features)
arousal_predictions = araousel_lstm_model.predict(arousal_features)

9360/9360 [==============================] - 9s 925us/step
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective window size : 1.000 (s)
Creating RawArray with float64 data, n_channels=12, n_times=7680
    Range : 0 ... 7679 =      0.000 ...    59.992 secs
Ready.
Effective

In [56]:
combined_predictions = []
for i in range(len(valence_predictions)):
    valence_pred = 1 if valence_predictions[i][0] > 0.5 else 0
    arousal_pred = 1 if arousal_predictions[i][0] > 0.5 else 0
    emotion = map_to_emotion(valence_pred, arousal_pred)
    combined_predictions.append(emotion)
    print(f"Valence: {valence_pred}, Arousal: {arousal_pred}, Emotion: {emotion}")

Valence: 1, Arousal: 1, Emotion: Happy
Valence: 1, Arousal: 0, Emotion: Content
Valence: 1, Arousal: 1, Emotion: Happy
Valence: 1, Arousal: 1, Emotion: Happy
Valence: 1, Arousal: 1, Emotion: Happy
Valence: 1, Arousal: 0, Emotion: Content
Valence: 1, Arousal: 0, Emotion: Content
Valence: 1, Arousal: 0, Emotion: Content
Valence: 0, Arousal: 0, Emotion: Sad
Valence: 1, Arousal: 0, Emotion: Content
Valence: 0, Arousal: 0, Emotion: Sad
Valence: 1, Arousal: 0, Emotion: Content
Valence: 0, Arousal: 0, Emotion: Sad
Valence: 0, Arousal: 1, Emotion: Angry
Valence: 1, Arousal: 1, Emotion: Happy
Valence: 1, Arousal: 1, Emotion: Happy
Valence: 0, Arousal: 0, Emotion: Sad
Valence: 1, Arousal: 1, Emotion: Happy
Valence: 0, Arousal: 1, Emotion: Angry
Valence: 1, Arousal: 1, Emotion: Happy
Valence: 1, Arousal: 0, Emotion: Content
Valence: 1, Arousal: 0, Emotion: Content
Valence: 1, Arousal: 1, Emotion: Happy
Valence: 1, Arousal: 1, Emotion: Happy
Valence: 1, Arousal: 1, Emotion: Happy
Valence: 1, Arous